In [1]:
import os
import sys
sys.path.append("../GradVisV2/toolbox")
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
import time
import colorama
from matplotlib import animation
import Visualization as vis
import nn_model
import trajectory_plots as tplot

In [2]:
#Random seed initialization
seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)


In [3]:
class ODEsolver():
    
    def __init__(self, x, initial_condition, epochs, architecture, initializer, activation, optimizer, prediction_save):
        """
        x : training domain (ex: x = np.linspace(0, 1, 100))
        initial_condition : initial condition including x0 and y0 (ex: initial_condition = (x0 = 0, y0 = 1))
        architecture : number of nodes in hidden layers (ex: architecture = [10, 10])
        initializer : weight initializer (ex: 'GlorotNormal')
        activation : activation function (ex: tf.nn.sigmoid)
        optimizer : minimization optimizer including parameters (ex: tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.5, beta_2 = 0.5, epsilon = 1e-07))
        prediciton_save : bool to save predicitons at each epoch during training (ex: prediction_save = False)
        """
        colorama.init()
        self.GREEN = colorama.Fore.GREEN
        self.RESET = colorama.Fore.RESET
        tf.keras.backend.set_floatx('float64')
        self.x = x
        self.initial_condition = initial_condition
        self.n = len(self.x)
        self.epochs = epochs
        self.architecture = architecture
        self.initializer = initializer
        self.activation = activation
        self.optimizer = optimizer
        self.neural_net = self.build_model()#self.neural_net_model(show = True)
        self.neural_net.summary()
        
        #Compile the model
        x = self.x
        x = tf.convert_to_tensor(x)
        x = tf.reshape(x, (self.n, 1))
        self.neural_net.compile(loss = self.custom_cost(x), optimizer = self.optimizer, experimental_run_tf_function = False)
        print("------- Model compiled -------")
        if prediction_save:
            self.predictions = []
        
        
        
    def build_model(self):
        """
        Builds a customized neural network model.
        """
        architecture = self.architecture
        initializer = self.initializer
        activation = self.activation
        
        nb_hidden_layers = len(architecture)
        input_tensor = tf.keras.layers.Input(shape = (1,))
        hidden_layers = []

        if nb_hidden_layers >= 1:
            hidden_layer = tf.keras.layers.Dense(architecture[0], kernel_initializer= initializer, bias_initializer='zeros',activation = activation)(input_tensor)
            hidden_layers.append(hidden_layer)
            for i in range(1, nb_hidden_layers):
                hidden_layer = tf.keras.layers.Dense(architecture[i], kernel_initializer= initializer, bias_initializer='zeros',activation = activation)(hidden_layers[i-1])
                hidden_layers.append(hidden_layer)
            output_layer = tf.keras.layers.Dense(1, kernel_initializer= initializer, bias_initializer = 'zeros', activation = tf.identity)(hidden_layers[-1])
        else:
            output_layer = tf.keras.layers.Dense(1, kernel_initializer= initializer, bias_initializer = 'zeros', activation = tf.identity)(input_tensor)
        
        model = tf.keras.Model(inputs = input_tensor, outputs = output_layer)
        return model
    
    
    @tf.function
    def NN_output(self, x):
        """
        x : must be of shape = (?, 1)
        Returns the output of the neural net
        """
        y = self.neural_net(x)
        return y
    

    def y_gradients(self, x):
        """
        Computes the gradient of y.
        """
        with tf.GradientTape() as tape1:
            tape1.watch(x)
            with tf.GradientTape() as tape2:
                tape2.watch(x)
                y = self.NN_output(x)
            dy_dx = tape2.gradient(y, x)
        d2y_dx2 = tape1.gradient(dy_dx,x)
        return y, dy_dx, d2y_dx2

    
    def differential_cost(self, x):
        """
        Defines the differential cost function for one neural network
        input.
        """
        y, dy_dx, d2y_dx2 = self.y_gradients(x)

        #----------------------------------------------
        #------------DIFFERENTIAL-EQUATION-------------
        #----------------------------------------------
        differential_equation = dy_dx + y - tf.math.exp(-x) * tf.math.cos(x)
        #----------------------------------------------
        #----------------------------------------------
        #----------------------------------------------

        return tf.square(differential_equation)


    def custom_cost(self, x):
        """
        Defines the cost function for a batch.
        """
        x0 = self.initial_condition[0]
        y0 = self.initial_condition[1]
        def loss(y_true, y_pred):
            differential_cost_term = tf.math.reduce_sum(self.differential_cost(x))
            boundary_cost_term = tf.square(self.NN_output(np.asarray([[x0]]))[0][0] - y0)
            return differential_cost_term/self.n + boundary_cost_term
        return loss
    
    def train(self):
        """
        neural_net : The built neural network returned by self.neural_net_model
        Trains the model according to x.
        """
        x = self.x
        x = tf.convert_to_tensor(x)
        x = tf.reshape(x, (self.n, 1))
        neural_net = self.neural_net
        
        if prediction_save:
            predictions = self.predictions

            #Define custom callback for predictions during training
            class PredictionCallback(tf.keras.callbacks.Callback):
                def on_epoch_end(self, epoch, logs={}):
                    y_predict = neural_net.predict(x)
                    predictions.append(y_predict)
                    print('Prediction saved at epoch: {}'.format(epoch))

            start_time = time.time()
            history = neural_net.fit(x = x, y = x, batch_size = self.n, epochs = self.epochs, callbacks = [PredictionCallback()])
            print(f"{self.GREEN}---   %s seconds ---  " % (time.time() - start_time))
            print(f"{self.RESET}")
            predictions = tf.reshape(predictions, (self.epochs, self.n))
            
        else:
            start_time = time.time()
            history = neural_net.fit(x = x, y = x, batch_size = self.n, epochs = self.epochs)
            print(f"{self.GREEN}---   %s seconds ---  " % (time.time() - start_time))
            print(f"{self.RESET}")
            
        return history
    
    
    def get_loss(self, history):
        """
        history : history of the training procedure returned by self.train
        Returns epochs and loss
        """
        epochs = history.epoch
        loss = history.history["loss"]
        return epochs, loss
    
    
    def predict(self, x_predict):
        """
        x_predict : domain of prediction (ex: x_predict = np.linspace(0, 1, 100))
        """
        domain_length = len(x_predict)
        x_predict = tf.convert_to_tensor(x_predict)
        x = tf.reshape(x_predict, (domain_length, 1))
        y_predict = self.neural_net.predict(x_predict)
        return y_predict


    def relative_error(self, y_predict, y_exact):
        """
        y_predict : array of predicted solution
        y_exact : array of exact solution
        Returns the relative error of the neural network solution
        given the exact solution.
        """
        if len(y_exact) != len(y_predict):
            raise Exception("y_predict and y_exact do not have the same shape.")
        relative_error = tf.keras.losses.mean_absolute_error(y_exact, y_predict) 
        return relative_error


    def MAE(self, y_predict, y_exact):
        """
        y_predict : array of predicted solution
        y_exact : array of exact solution
        Returns the mean absolute error of the neural network solution
        given the exact solution.
        """
        if len(y_exact) != len(y_predict):
            raise Exception("y_predict and y_exact do not have the same shape.")
        mae = tf.keras.losses.MAE(y_exact, y_predict)
        return mae
    
    
    def get_predictions(self):
        """
        Returns the neural net predictions at each epoch 
        """
        if not prediction_save:
            raise Exception("The predictions have not been saved.")
        else:
            return self.predictions
        
        
    def training_animation(self, y_exact, y_predict, epoch, loss):
        """
        Plot the training animation including the exact solution, 
        the neural network solution and the cost function as functions of epochs.
        This function needs the model to be trained and requires the outputs of get_loss.
        """
        if not prediction_save:
            raise Exception("The predictions have not been saved.")
        fig, ax = plt.subplots()
        ax1 = plt.axes()
        ax2 = fig.add_axes([0.58, 0.2, 0.3, 0.2])

        frames = []

        x = self.x
        predictions = self.predictions

        x_loss = epoch
        y_loss = loss

        ax1.plot(x, y_exact, "C1", label = "Exact solution")
        ax1.legend(loc = 'upper left')
        ax1.set_xlim(min(x), max(x))
        ax1.set_ylim(min(y_predict) - 0.1, max(y_predict) + 0.1)
        ax1.set_xlabel("$x$", fontsize = 15)
        ax1.set_ylabel("$\hat{f}$", fontsize = 15)

        ax2.set_xlabel("Epochs")
        ax2.set_ylabel("Loss")
        ax2.semilogy(x_loss, y_loss, color = "w", linewidth = 1.0)

        x_loss_points = []
        y_loss_points = []

        for i in range(self.epochs):
            y = predictions[i]
            frame1, = ax1.plot(x, y, ".", color = "C0")
            x_loss_points.append(x_loss[i])
            y_loss_points.append(y_loss[i])
            frame2, = ax2.semilogy(x_loss_points, y_loss_points, color = "C0")
            frames.append([frame1, frame2])

        ani = animation.ArtistAnimation(fig, frames, interval = 20, blit = True)
        # Set up formatting for the movie files
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        ani.save('./data/animation.mp4', writer=writer)
        #plt.show()
        
    

In [4]:
if __name__ == "__main__":

        #--------------------------------------------------------------------
        #-----------------PARAMETER-INITIALIZATION---------------------------
        #--------------------------------------------------------------------

        #Training domain
        x = np.linspace(0, 1, 100)
        #Initial conditions
        initial_condition = (0, 0)
        #Number of epochs
        epochs = 200
        #Structure of the neural net (only hidden layers)
        architecture = [10]
        #Initializer used
        initializer = 'GlorotNormal'
        #Activation function used
        activation = tf.nn.sigmoid
        #Optimizer used
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.5, beta_2 = 0.5, epsilon = 1e-07)
        #Save predictions at each epoch
        prediction_save = False

        #--------------------------------------------------------------------
        #------------------MODEL-DEFINITION-AND-TRAINING---------------------
        #--------------------------------------------------------------------

        #Class definition
        solver = ODEsolver(x, initial_condition, epochs, architecture, initializer, activation, optimizer, prediction_save)
        #Training
        history = solver.train()
        epoch, loss = solver.get_loss(history)

        #--------------------------------------------------------------------
        #------------------PREDICTION----------------------------------------
        #--------------------------------------------------------------------

        #Plot the exact and the neural net solution
        x_predict = x
        y_predict = solver.predict(x_predict)
        y_exact = np.exp(-x_predict)*np.sin(x_predict)
        #plt.plot(x_predict, y_exact, label = "Exact solution")
        #plt.plot(x_predict, y_predict, ".", label = "Neural network solution")
        #plt.legend()
        #plt.savefig("./data/prediction.pdf")

        #Plot the relative error
        relative_error = solver.relative_error(y_predict, y_exact)
        #plt.semilogy(x_predict, relative_error)
        #plt.show()

        #--------------------------------------------------------------------
        #------------------TRAINING-ANIMATION--------------------------------
        #--------------------------------------------------------------------

        #solver.training_animation(y_exact, y_predict, epoch, loss)

        #--------------------------------------------------------------------
        #------------------LOSS-SURFACE--------------------------------
        #--------------------------------------------------------------------
        solver.neural_net.save_weights('./data/minimum_0')
        nnmodel = nn_model.Tensorflow_NNModel(solver.neural_net, solver.neural_net.loss, solver.x, './data/minimum_0')
        #vis.visualize(nnmodel,solver.neural_net.loss,solver.x, './data/minimum_0', 165, './data/example',random_dir = True, proz = 1.5, verbose=True)
        #tplot.plot_loss_2D('./data/example.npz','./data/plot2d',is_log=False)
        #plot.plot_loss_3D('./data/example.npz','./data/plot3d',is_log=False, degrees = 120)
        #outs = np.load('./data/example.npz', allow_pickle=True)
        #outs = outs["a"]
        #tplot.plot3D(outs[0][0],outs[0][1],outs[0][2], './data/interactive_plot.html')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                20        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________
------- Model compiled -------
Epoch 1/200
1/1 [==============================] - ETA: 0s - loss: 1.158 - 0s 2ms/step - loss: 1.1580
Epoch 2/200
1/1 [==============================] - ETA: 0s - loss: 1.139 - 0s 17ms/step - loss: 1.1397
Epoch 3/200
1/1 [==============================] - ETA: 0s - loss: 1.121 - 0s 19ms/step - loss: 1.1216
Epoch 4/200
1/1 [==============================] - ETA: 

Epoch 73/200
1/1 [==============================] - ETA: 0s - loss: 0.371 - 0s 4ms/step - loss: 0.3716
Epoch 74/200
1/1 [==============================] - ETA: 0s - loss: 0.367 - 0s 4ms/step - loss: 0.3679
Epoch 75/200
1/1 [==============================] - ETA: 0s - loss: 0.364 - 0s 4ms/step - loss: 0.3644
Epoch 76/200
1/1 [==============================] - ETA: 0s - loss: 0.361 - 0s 3ms/step - loss: 0.3611
Epoch 77/200
1/1 [==============================] - ETA: 0s - loss: 0.357 - 0s 3ms/step - loss: 0.3579
Epoch 78/200
1/1 [==============================] - ETA: 0s - loss: 0.354 - 0s 9ms/step - loss: 0.3548
Epoch 79/200
1/1 [==============================] - ETA: 0s - loss: 0.351 - 0s 9ms/step - loss: 0.3519
Epoch 80/200
1/1 [==============================] - ETA: 0s - loss: 0.349 - 0s 3ms/step - loss: 0.3491
Epoch 81/200
1/1 [==============================] - ETA: 0s - loss: 0.346 - 0s 13ms/step - loss: 0.3463
Epoch 82/200
1/1 [==============================] - ETA: 0s - loss: 0.34

Epoch 152/200
1/1 [==============================] - ETA: 0s - loss: 0.238 - 0s 2ms/step - loss: 0.2380
Epoch 153/200
1/1 [==============================] - ETA: 0s - loss: 0.236 - 0s 3ms/step - loss: 0.2368
Epoch 154/200
1/1 [==============================] - ETA: 0s - loss: 0.235 - 0s 7ms/step - loss: 0.2356
Epoch 155/200
1/1 [==============================] - ETA: 0s - loss: 0.234 - 0s 3ms/step - loss: 0.2344
Epoch 156/200
1/1 [==============================] - ETA: 0s - loss: 0.233 - 0s 4ms/step - loss: 0.2332
Epoch 157/200
1/1 [==============================] - ETA: 0s - loss: 0.232 - 0s 4ms/step - loss: 0.2321
Epoch 158/200
1/1 [==============================] - ETA: 0s - loss: 0.230 - 0s 3ms/step - loss: 0.2309
Epoch 159/200
1/1 [==============================] - ETA: 0s - loss: 0.229 - 0s 3ms/step - loss: 0.2297
Epoch 160/200
1/1 [==============================] - ETA: 0s - loss: 0.228 - 0s 3ms/step - loss: 0.2285
Epoch 161/200
1/1 [==============================] - ETA: 0s - l

In [5]:
vis.visualize(nnmodel,solver.neural_net.loss,solver.x, ['./data/minimum_0'], 20, './data/example',random_dir = False, proz = 1.5, verbose=True)
#tplot.plot_loss_2D('./data/example.npz','./data/plot2d',is_log=False)
#plot.plot_loss_3D('./data/example.npz','./data/plot3d',is_log=False, degrees = 120)
#outs = np.load('./data/example.npz', allow_pickle=True)
#outs = outs["a"]
#tplot.plot3D(outs[0][0],outs[0][1],outs[0][2], './data/interactive_plot.html')

#np.min(outs[0][2])

Calculating PCA directions...
Principal [1.00000000e+00 9.66195564e-33]
Normalizing...
Calculating coefficients...
Calculating Z-values of paths...


  0%|          | 0/1 [00:00<?, ?it/s]

Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)


100%|██████████| 1/1 [00:00<00:00, 153.26it/s]


Calculating loss landscape...


...

 90%|█████████ | 18/20 [00:03<00:00,  5.77it/s]

Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.18489055

 95%|█████████▌| 19/20 [00:03<00:00,  6.00it/s]

Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.1848905571512366, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.18489055

100%|██████████| 20/20 [00:03<00:00,  5.59it/s]


In [7]:
outs = np.load('./data/example.npz', allow_pickle=True)
outs = outs["a"]

tplot.plot3D(outs[0][0],outs[0][1],outs[0][2], './data/interactive_plot.html')

/usr/lib/python3.8/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [9]:
tplot.plot3D(outs[0][0],outs[0][1],outs[0][2])

In [7]:
np.linspace(-7.5,7.5,165)

array([-7.5       , -7.40853659, -7.31707317, -7.22560976, -7.13414634,
       -7.04268293, -6.95121951, -6.8597561 , -6.76829268, -6.67682927,
       -6.58536585, -6.49390244, -6.40243902, -6.31097561, -6.2195122 ,
       -6.12804878, -6.03658537, -5.94512195, -5.85365854, -5.76219512,
       -5.67073171, -5.57926829, -5.48780488, -5.39634146, -5.30487805,
       -5.21341463, -5.12195122, -5.0304878 , -4.93902439, -4.84756098,
       -4.75609756, -4.66463415, -4.57317073, -4.48170732, -4.3902439 ,
       -4.29878049, -4.20731707, -4.11585366, -4.02439024, -3.93292683,
       -3.84146341, -3.75      , -3.65853659, -3.56707317, -3.47560976,
       -3.38414634, -3.29268293, -3.20121951, -3.1097561 , -3.01829268,
       -2.92682927, -2.83536585, -2.74390244, -2.65243902, -2.56097561,
       -2.4695122 , -2.37804878, -2.28658537, -2.19512195, -2.10365854,
       -2.01219512, -1.92073171, -1.82926829, -1.73780488, -1.64634146,
       -1.55487805, -1.46341463, -1.37195122, -1.2804878 , -1.18